In [ ]:
%uv sync

In [ ]:
from dotenv import load_dotenv
from azure.search.documents.indexes.aio import SearchIndexClient
from azure.core.credentials import AzureKeyCredential
from azure.search.documents.indexes.models import (
    SearchField,
    VectorSearch,
    HnswAlgorithmConfiguration,
    VectorSearchProfile,
    SearchIndex,    
    SearchFieldDataType
)
import os

In [4]:
load_dotenv(override=True)

search_endpoint = os.getenv('SEARCH_ENDPOINT')
search_api_key = os.getenv('SEARCH_API_KEY')

In [ ]:
index = {
        'name': 'lore',
        'fields': [
                SearchField(name="id", type=SearchFieldDataType.String,key=True),                   
                SearchField(name="type", type=SearchFieldDataType.String, searchable=True,sortable=False, facetable=True, filterable=False),                      
                SearchField(name="title", type=SearchFieldDataType.String, searchable=True,sortable=False, facetable=True, filterable=False),                  
                SearchField(name="content", type=SearchFieldDataType.String, searchable=True,sortable=False, facetable=False, filterable=False),                
                SearchField(name="content_vector", type=SearchFieldDataType.Collection(SearchFieldDataType.Single), vector_search_dimensions=1024, vector_search_profile_name="vector-profile-1",searchable=True,sortable=False, facetable=False, filterable=False),
                SearchField(name="category", type=SearchFieldDataType.String, searchable=True,sortable=False, facetable=True, filterable=False)    
        ]    
} 
